# TODOs

- ✓ def solve => def get_instance, returner problem i stedet for løsning. Bl.a. mulighed for at få feasibility.
- ✓ def get_task_counts, laver stats matrix
- ✓ arg sick => arg hu + arg sick
- ✓ arg df => arg task_counts
- ✓ arg assigned
- ✓ løs for hel dag
- ✓ tilføj constraints i prob
- ✓ tilføj allerede assigned constraint
- ✓ tilføj vagthavende constraint


In [7]:
import pandas as pd
import numpy as np
import pulp as P

from georgstage.solver import Task, team_minmax, get_task_counts, print_sol, get_instance

## Create synthetic data

In [4]:
MAX_SICK = 2
N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]
N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
N_TASKS = len(Task)
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]
KABYS_TIDER = [0,1,2]
PEJLE_SHIFT = 3


dates = pd.date_range(start='3/1/2021', end='4/14/2021').date

history = []
np.random.seed(42)

for dt in dates:
    for shift in range(N_SHIFTS):
        n_sick = np.random.randint(0,MAX_SICK)
        sick = np.random.choice(N_GASTS, n_sick)
        team = TEAM_ORDER[shift % N_TEAMS]
        min_gast, max_gast = team_minmax(team)
        active_gasts = [gast for gast in range(min_gast, max_gast) if gast not in sick]
        for task_no, gast_no in enumerate(np.random.permutation(active_gasts)[:len(Task)]):
            task_name = Task(task_no).name
                entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': task_no, 'task_name': task_name}
            history.append(entry)
        for gast_no in sick:
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': -1, 'task_name': 'SICK'}
            history.append(entry)
df = pd.DataFrame(history)
df.to_csv('../data/synthetic.csv', index=False)

## Load data

In [6]:
df = pd.read_csv('../data/synthetic.csv')
df

,date,shift,gast_no,task_no,task_name
0,2021-03-01,0,0,0,VAGTHAVENDE_ELEV
1,2021-03-01,0,17,1,ORDONNANS
2,2021-03-01,0,15,2,UDKIG
3,2021-03-01,0,1,3,BJÆRGEMÆRS
4,2021-03-01,0,8,4,RORGÆNGER
...,...,...,...,...,...
3646,2021-04-14,5,55,9,UDSÆTNINGSGAST_E
3647,2021-04-14,5,57,10,PEJLEGAST_A
3648,2021-04-14,5,41,11,PEJLEGAST_B
3649,2021-04-14,5,50,12,DÆKSELEV_I_KABYS


## Example

In [8]:
# simulate som sick gasts
TIDSPUNKTER = ['08 - 12', '12 - 16', '16 - 20', '20 - 24', '24 - 04', '04 - 08']

task_counts = get_task_counts(df)

n_gasts, n_tasks = task_counts.shape
n_shifts = 6

sick = [2,10]
hu = [4,5,8]
assigned = {0: {6: Task.ORDONNANS, 3: Task.BJÆRGEMÆRS}}

#prob, X = get_instance(task_counts, [1,21,41], pejlegast_b=52, sick=sick, hu=hu, assigned=assigned)
prob, X = get_instance(task_counts, [1,21,41], pejlegast_b=52, assigned=assigned, hu=hu, sick=sick)

status = prob.solve()
print(f'Optimal? {"yes" if status == 1 else "no"}')
print(f'Infeasible? {"yes" if status == -1 else "no"}')
print()

for k in range(n_shifts):
    print(f'Vagt {TIDSPUNKTER[k]}')
    for j in range(len(Task)):
        for i in range(n_gasts):
            x = X[i][j][k]
            if x.varValue == 1:
                task_name = Task(j)
                count = task_counts[i][j]
                print(f'- Gast {i} er {task_name} ({count} før)')



Optimal? yes
Infeasible? no

Vagt 08 - 12
- Gast 1 er Task.VAGTHAVENDE_ELEV (1 før)
- Gast 6 er Task.ORDONNANS (3 før)
- Gast 12 er Task.UDKIG (3 før)
- Gast 3 er Task.BJÆRGEMÆRS (3 før)
- Gast 18 er Task.RORGÆNGER (2 før)
- Gast 16 er Task.UDSÆTNINGSGAST_A (2 før)
- Gast 13 er Task.UDSÆTNINGSGAST_B (1 før)
- Gast 15 er Task.UDSÆTNINGSGAST_C (1 før)
- Gast 9 er Task.UDSÆTNINGSGAST_D (3 før)
- Gast 14 er Task.UDSÆTNINGSGAST_E (2 før)
- Gast 7 er Task.DÆKSELEV_I_KABYS (3 før)
Vagt 12 - 16
- Gast 21 er Task.VAGTHAVENDE_ELEV (5 før)
- Gast 22 er Task.ORDONNANS (1 før)
- Gast 24 er Task.UDKIG (1 før)
- Gast 29 er Task.BJÆRGEMÆRS (1 før)
- Gast 35 er Task.RORGÆNGER (0 før)
- Gast 30 er Task.UDSÆTNINGSGAST_A (1 før)
- Gast 36 er Task.UDSÆTNINGSGAST_B (1 før)
- Gast 31 er Task.UDSÆTNINGSGAST_C (2 før)
- Gast 26 er Task.UDSÆTNINGSGAST_D (1 før)
- Gast 37 er Task.UDSÆTNINGSGAST_E (1 før)
- Gast 20 er Task.DÆKSELEV_I_KABYS (2 før)
Vagt 16 - 20
- Gast 41 er Task.VAGTHAVENDE_ELEV (12 før)
- Gast 48

In [14]:
task_counts[:20]

array([[ 3,  3,  5,  4,  4,  4,  4,  7,  6,  4,  6,  3,  5],
       [ 1,  5,  5,  4,  6,  2,  5,  1,  5,  6,  5,  3,  5],
       [ 7,  5,  2,  6,  6,  3,  1,  2,  3,  6,  1, 10,  4],
       [ 4,  5,  6,  3,  5,  9,  7,  5,  2,  5,  4,  1,  4],
       [ 2,  2,  6,  7,  5,  3,  6,  7,  5,  4,  2,  4,  6],
       [ 9,  3,  3,  2,  6,  2,  3,  1,  6,  2,  2,  9,  4],
       [ 6,  3,  6,  6,  3,  2,  4,  7,  6,  3,  7,  3,  3],
       [ 4,  5,  4,  5,  4,  2,  3,  7,  5,  7,  3,  7,  3],
       [ 4,  4,  2,  5,  7,  2,  5,  9,  3,  3,  2,  2,  4],
       [ 5,  5,  4,  3,  3,  7,  3,  3,  3,  2, 10,  8,  6],
       [ 9,  4,  3,  8,  3,  7,  7,  9,  3,  8,  3,  4,  1],
       [ 1,  3,  7,  3,  3,  3,  9,  4,  8,  5,  7,  1,  6],
       [ 4,  7,  3,  6,  2,  4,  3,  4,  6,  6,  7,  2,  4],
       [ 5,  6,  5,  5,  6,  5,  1,  6,  4,  6,  2,  6,  5],
       [ 7,  4,  6,  1,  6,  6,  7,  3,  3,  2,  3,  8,  4],
       [ 3,  8,  6,  4,  3, 11,  2,  1,  4, 10,  6,  3,  3],
       [ 6,  5,  5,  4, 